In [7]:
import keras

from sacred import Experiment
from sacred.observers import MongoObserver, FileStorageObserver

ex = Experiment("notebook-keras-sacred", interactive=True)

ex.observers.append(FileStorageObserver("cnn2-experiment"))
# Uncomment to add MongoDB support 
#ex.observers.append(MongoObserver(url="mongodb://<user>:<password>@<hostname>:<port>/<auth-db>", db_name="<sacred-db>"))


@ex.config
def cfg():
    batch_size = 64
    num_epochs = 10
    learning_rate = 0.01


class MetricsLoggerCallback(keras.callbacks.Callback):
    def __init__(self, _run):
        super().__init__()
        self._run = _run

    def on_epoch_end(self, _, logs):
        self._run.log_scalar("training.loss", logs.get('loss'))
        self._run.log_scalar("training.acc", logs.get('accuracy'))
        self._run.log_scalar("validation.loss", logs.get('val_loss'))
        self._run.log_scalar("validation.acc", logs.get('val_accuracy'))

@ex.main
def run(_run, batch_size, num_epochs, learning_rate):
    
    # importing the dataset
    fashion_mnist = keras.datasets.fashion_mnist
    (train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

    # normalizing the images to 0 and 1
    train_images = train_images / 255.0
    test_images = test_images / 255.0

    # setting up the model
    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(28, 28)),
        keras.layers.Dense(128, activation='relu'),
        keras.layers.Dense(10, activation='softmax')
    ])

    # compiling the model
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

    # training the model
    model.fit(train_images, train_labels,
            batch_size=batch_size,
            epochs=num_epochs,
            validation_data=(test_images, test_labels),
            callbacks=[MetricsLoggerCallback(_run)],
            verbose=2)

    # evaluating the model
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    ex.log_scalar("test loss", test_loss)
    ex.log_scalar("test acc", test_acc)
    print('Test loss:', test_loss, ' and Test accuracy:', test_acc)
    
    # We also can store our trained model
    # pickle.dump(model, open("mymodel.p", "wb"))
    # ex.add_artifact("mymodel.p")

    
    return test_acc


In [8]:
ex
ex.run()

INFO - notebook-keras-sacred - Running command 'run'
INFO - notebook-keras-sacred - Started run with ID "8"


Epoch 1/10
938/938 - 1s - loss: 1.7796 - accuracy: 0.6805 - val_loss: 1.7307 - val_accuracy: 0.7299
Epoch 2/10
938/938 - 1s - loss: 1.7137 - accuracy: 0.7468 - val_loss: 1.7144 - val_accuracy: 0.7458
Epoch 3/10
938/938 - 1s - loss: 1.7097 - accuracy: 0.7509 - val_loss: 1.7227 - val_accuracy: 0.7379
Epoch 4/10
938/938 - 1s - loss: 1.7070 - accuracy: 0.7536 - val_loss: 1.6955 - val_accuracy: 0.7653
Epoch 5/10
938/938 - 1s - loss: 1.7017 - accuracy: 0.7589 - val_loss: 1.7358 - val_accuracy: 0.7248
Epoch 6/10
938/938 - 1s - loss: 1.7039 - accuracy: 0.7569 - val_loss: 1.7014 - val_accuracy: 0.7597
Epoch 7/10
938/938 - 1s - loss: 1.7075 - accuracy: 0.7533 - val_loss: 1.7075 - val_accuracy: 0.7537
Epoch 8/10
938/938 - 1s - loss: 1.7030 - accuracy: 0.7579 - val_loss: 1.7133 - val_accuracy: 0.7477
Epoch 9/10
938/938 - 1s - loss: 1.7141 - accuracy: 0.7469 - val_loss: 1.7299 - val_accuracy: 0.7316
Epoch 10/10
938/938 - 1s - loss: 1.7030 - accuracy: 0.7579 - val_loss: 1.7029 - val_accuracy: 0.7583

INFO - notebook-keras-sacred - Result: 0.7583000063896179
INFO - notebook-keras-sacred - Completed after 0:00:12


Test loss: 1.7028793096542358  and Test accuracy: 0.7583000063896179
